Todos:
* remove the hardcoded values
* see if I can batch the training, so I don't have to choose between a subset of data or a memory error*
* add the ability to input a text sentence and get back a response
* save the model to an h5 file for inference

### Neural Machine Translation using word level language model and embeddings in Keras

In [154]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split


In [177]:
MINI_BATCH_SIZE=1000

In [155]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [156]:
lines= pd.read_table('output1.txt', names=['inp', 'outp'])

In [157]:
#lines = lines[0:1000]

In [158]:
lines.shape

(7840, 2)

In [159]:
lines.sample(10)

,inp,outp
7465,AtAmazonHelp I cancelled my prime account 3 ti...,AtCustomerName My apologies for the unexpected...
1332,@UPSHelp AtAmazonHelp I AM NOT PICKING THIS PA...,AtCustomerName I'm sorry to hear about this ex...
5700,AtAmazonHelp Can you please conclude so that I...,AtCustomerName I get you're upset regarding yo...
7090,AtAmazonHelp December the 3rd it still shows a...,AtCustomerName I'm very sorry for the inconven...
7440,Met an Amazon delivery man in the elevator. He...,AtCustomerName We deliver Christmas trees and ...
5280,AtAmazonHelp I had pre-booked one plus 6t.init...,AtCustomerName I understand your concern regar...
6917,AtAmazonHelp I just placed an order on that I ...,"AtCustomerName Hey there, Fred. Could you plea..."
914,AtAmazonHelp AtAmazonHelp AtAmazonHelpIN @amit...,AtCustomerName I get your disappointment. We’d...
310,AtAmazonHelp I presume UK,AtCustomerName Did you get the chance to view ...
507,My favourite recent discovery? The error mes...,AtCustomerName It is Great !!!


#### Data Cleanup

In [160]:
lines.inp=lines.inp.apply(lambda x: x.lower())
lines.outp=lines.outp.apply(lambda x: x.lower())

In [161]:
# Take the length as 50
lines.inp=lines.inp.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.outp=lines.outp.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [162]:
exclude = set(string.punctuation)
lines.inp=lines.inp.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.outp=lines.outp.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [163]:
#remove_digits = str.maketrans('', '', digits)
#lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
#lines.fr=lines.fr.apply(lambda x: x.translate(remove_digits))

In [164]:
lines.sample(10)

,inp,outp
3868,atamazonhelp i’ve called you help support and ...,atcustomername wed like to make sure this is t...
6508,atamazonhelpecho hello COMMA can you please ad...,atcustomername thats odd wed like to live trou...
4477,really atamazonhelp your delivery people take ...,atcustomername wed like to get this escalated ...
4004,atamazonhelp hi i ordered some lego as a prese...,atcustomername i do apologize for the way your...
796,atamazonhelp can you please tell me the custom...,atcustomername hey there we can be reached 247...
7050,atamazonhelp atamazonhelp not happy i won’t ge...,atcustomername very sorry to hear your order i...
2722,atamazonhelp plus this isnt the first time del...,atcustomername apologies the seller should re...
3324,atamazonhelp just tell me what to do when i fo...,atcustomername please try resetting your passw...
7786,atamazonhelp i really love your service COMMA...,atcustomername thanks for sharing your feedbac...
2019,atamazonhelp im trying to avoid spending money...,atcustomername the release date for this item ...


#### Generate synthetic data

In [165]:
lines.outp = lines.outp.apply(lambda x : 'START_ '+ x + ' _END')

In [166]:
all_words=set()
for inp in lines.inp:
    for word in inp.split():
        if word not in all_words:
            all_words.add(word)
    
all_outp_words=set()
for outp in lines.outp:
    for word in outp.split():
        if word not in all_words:
            all_words.add(word)

In [167]:
len(all_words)

15081

In [168]:
length_list=[]
for l in lines.inp:
    length_list.append(len(l.split(' ')))
max_input_length = np.max(length_list)
max_input_length

72

In [169]:
length_list=[]
for l in lines.outp:
    length_list.append(len(l.split(' ')))
max_output_length = np.max(length_list)
max_output_length

61

In [170]:
words = sorted(list(all_words))
num_tokens = len(all_words)


In [171]:
token_index = dict(
    [(word, i) for i, word in enumerate(all_words)])


In [172]:
len(lines.outp)*max_input_length*num_tokens

8512922880

In [178]:
encoder_input_data = np.zeros(
    (len(lines.inp[:MINI_BATCH_SIZE]), max_input_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.outp[:MINI_BATCH_SIZE]), max_output_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.outp[:MINI_BATCH_SIZE]), max_output_length, num_tokens),
    dtype='float32')

In [179]:
len(lines.outp[:1000])

1000

In [180]:
for i, (input_text, target_text) in enumerate(zip(lines.inp[:MINI_BATCH_SIZE], lines.outp[:MINI_BATCH_SIZE])):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = token_index[word]

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, token_index[word]] = 1.

### Build keras encoder-decoder model

In [181]:
embedding_size = max_input_length

In [182]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

In [183]:
import keras
print(keras.__version__)

2.2.4


#### Encoder model

In [184]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(max_input_length, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#### Decoder model

In [185]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(max_input_length, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


In [186]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 72)     1085832     input_13[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 72)     1085832     input_14[0][0]                   
__________________________________________________________________________________________________
lstm_9 (LS

#### Fit the model

In [187]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=200,
          validation_split=0.05)

Train on 950 samples, validate on 50 samples
Epoch 1/200
950/950 [==============================] - 75s 79ms/step - loss: 4.4754 - acc: 0.0275 - val_loss: 4.1108 - val_acc: 0.0243
Epoch 2/200
950/950 [==============================] - 65s 68ms/step - loss: 3.6945 - acc: 0.0252 - val_loss: 3.3360 - val_acc: 0.0197
Epoch 3/200
950/950 [==============================] - 64s 68ms/step - loss: 3.0819 - acc: 0.0172 - val_loss: 2.9612 - val_acc: 0.0197
Epoch 4/200
950/950 [==============================] - 64s 67ms/step - loss: 2.8355 - acc: 0.0172 - val_loss: 2.8142 - val_acc: 0.0197
Epoch 5/200
950/950 [==============================] - 63s 67ms/step - loss: 2.7258 - acc: 0.0177 - val_loss: 2.7387 - val_acc: 0.0187
Epoch 6/200
950/950 [==============================] - 64s 68ms/step - loss: 2.6677 - acc: 0.0214 - val_loss: 2.7030 - val_acc: 0.0200
Epoch 7/200
950/950 [==============================] - 64s 67ms/step - loss: 2.6354 - acc: 0.0359 - val_loss: 2.6832 - val_acc: 0.0364
Epoch 8/20

Epoch 61/200
950/950 [==============================] - 64s 67ms/step - loss: 1.9280 - acc: 0.0986 - val_loss: 2.1240 - val_acc: 0.0934
Epoch 62/200
950/950 [==============================] - 63s 66ms/step - loss: 1.9165 - acc: 0.1008 - val_loss: 2.1104 - val_acc: 0.0951
Epoch 63/200
950/950 [==============================] - 64s 67ms/step - loss: 1.9028 - acc: 0.1038 - val_loss: 2.1014 - val_acc: 0.0984
Epoch 64/200
950/950 [==============================] - 64s 67ms/step - loss: 1.8888 - acc: 0.1067 - val_loss: 2.0929 - val_acc: 0.1000
Epoch 65/200
950/950 [==============================] - 63s 67ms/step - loss: 1.8775 - acc: 0.1088 - val_loss: 2.0809 - val_acc: 0.1000
Epoch 66/200
950/950 [==============================] - 65s 69ms/step - loss: 1.8622 - acc: 0.1114 - val_loss: 2.0711 - val_acc: 0.1023
Epoch 67/200
950/950 [==============================] - 64s 68ms/step - loss: 1.8493 - acc: 0.1149 - val_loss: 2.0561 - val_acc: 0.1036
Epoch 68/200
950/950 [==========================

950/950 [==============================] - 63s 66ms/step - loss: 1.3038 - acc: 0.2033 - val_loss: 1.6591 - val_acc: 0.1862
Epoch 122/200
950/950 [==============================] - 63s 66ms/step - loss: 1.2958 - acc: 0.2043 - val_loss: 1.6615 - val_acc: 0.1872
Epoch 123/200
950/950 [==============================] - 63s 67ms/step - loss: 1.2904 - acc: 0.2052 - val_loss: 1.6518 - val_acc: 0.1892
Epoch 124/200
950/950 [==============================] - 63s 66ms/step - loss: 1.2818 - acc: 0.2066 - val_loss: 1.6536 - val_acc: 0.1895
Epoch 125/200
950/950 [==============================] - 63s 66ms/step - loss: 1.2764 - acc: 0.2067 - val_loss: 1.6480 - val_acc: 0.1911
Epoch 126/200
950/950 [==============================] - 63s 66ms/step - loss: 1.2703 - acc: 0.2075 - val_loss: 1.6447 - val_acc: 0.1902
Epoch 127/200
950/950 [==============================] - 63s 66ms/step - loss: 1.2626 - acc: 0.2094 - val_loss: 1.6376 - val_acc: 0.1918
Epoch 128/200
950/950 [==============================] 

Epoch 181/200
950/950 [==============================] - 63s 66ms/step - loss: 0.9828 - acc: 0.2505 - val_loss: 1.5506 - val_acc: 0.2075
Epoch 182/200
950/950 [==============================] - 64s 67ms/step - loss: 0.9811 - acc: 0.2512 - val_loss: 1.5696 - val_acc: 0.2052
Epoch 183/200
950/950 [==============================] - 63s 67ms/step - loss: 0.9744 - acc: 0.2523 - val_loss: 1.5624 - val_acc: 0.2069
Epoch 184/200
950/950 [==============================] - 63s 67ms/step - loss: 0.9718 - acc: 0.2529 - val_loss: 1.5561 - val_acc: 0.2043
Epoch 185/200
950/950 [==============================] - 63s 67ms/step - loss: 0.9669 - acc: 0.2539 - val_loss: 1.5560 - val_acc: 0.2069
Epoch 186/200
950/950 [==============================] - 64s 67ms/step - loss: 0.9621 - acc: 0.2538 - val_loss: 1.5569 - val_acc: 0.2069
Epoch 187/200
950/950 [==============================] - 63s 67ms/step - loss: 0.9615 - acc: 0.2537 - val_loss: 1.5601 - val_acc: 0.2098
Epoch 188/200
950/950 [==================

In [188]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()






decoder_state_input_h = Input(shape=(max_input_length,))
decoder_state_input_c = Input(shape=(max_input_length,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


with open('encoder_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('encoder_model_weights.h5')

with open('decoder_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('decoder_model_weights.h5')


decoder_model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_9 (Embedding)      (None, None, 72)          1085832   
_________________________________________________________________
lstm_9 (LSTM)                [(None, 72), (None, 72),  41760     
Total params: 1,127,592
Trainable params: 1,127,592
Non-trainable params: 0
_________________________________________________________________


/home/rich/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_10 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_15:0' shape=(?, 72) dtype=float32>, <tf.Tensor 'input_16:0' shape=(?, 72) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 72)     1085832     input_14[0][0]                   
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 72)           0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 72)           0                                            
__________________________________________________________________________________________________
lstm_10 (L

# INFERENCE MODE
#### Create sampling model

In [189]:
#TODO:  Load Models from disk
from keras.models import model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder_model = load_model('encoder_model.json', 'encoder_model_weights.h5')
decoder_model = load_model('decoder_model.json', 'decoder_model_weights.h5')
encoder_model.summary()
decoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_9 (Embedding)      (None, None, 72)          1085832   
_________________________________________________________________
lstm_9 (LSTM)                [(None, 72), (None, 72),  41760     
Total params: 1,127,592
Trainable params: 1,127,592
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________


In [190]:
reverse_char_index = dict(
    (i, char) for char, i in token_index.items())


#### Function to generate sequences

In [191]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_char_index[sampled_token_index]
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 180):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Look at the some translations

In [192]:
for seq_index in [100, 200, 300]:
    input_seq = encoder_input_data[seq_index]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.inp[seq_index])
    print('Decoded sentence:', decoded_sentence.replace(' COMMA', ','))
    print('Decoded sentence length: {}'.format(len(decoded_sentence.split(' '))))

-
Input sentence: atamazonhelp hi COMMA i ordered something on the 25th october and it’s still not arrived i’ve messaged the seller and it said they can’t receive my email i do not think this company is genuine how do i get a refund thanks
Decoded sentence:  atcustomername your order order will be able to take a closer look at this further appreciate your understanding pv _END
Decoded sentence length: 21
-
Input sentence: atamazonhelp hi there COMMA how long after returning damaged books to you via doddle on 201118 COMMA would i expect to see a change in my orders status and or refund it currently says return started  thank you jacq
Decoded sentence:  atcustomername your order order will be able to take a closer look at this further appreciate your understanding pv _END
Decoded sentence length: 21
-
Input sentence: atamazonhelp for you guys 12hour means  COMMA COMMA COMMA how many days  COMMA yesterday you guys told me that our back end reach you with in 12hr but now i completed 15 hou

In [200]:
input = np.zeros(72)
input_text = 'atamazonhelp where is my order i have been waiting for weeks to receive it and it still has not arrived'
for i, word in enumerate(input_text.split()):
        input[i] = token_index[word]

In [201]:
input

array([1.3387e+04, 1.3592e+04, 1.4329e+04, 8.7110e+03, 1.2226e+04,
       1.4295e+04, 5.7590e+03, 1.0188e+04, 1.4198e+04, 4.7390e+03,
       5.7060e+03, 1.1700e+03, 6.8560e+03, 9.9320e+03, 9.7300e+03,
       9.9320e+03, 8.1840e+03, 1.1000e+01, 2.5580e+03, 2.2500e+02,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00])

In [202]:
decoded_sentence = decode_sequence(input)

In [203]:
decoded_sentence

' atcustomername your order order will be able to take a closer look at this further appreciate your understanding pv _END'

In [199]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open('token_index.p', 'wb') as fp:
    pickle.dump(token_index, fp, protocol=pickle.HIGHEST_PROTOCOL)